In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from category_encoders.one_hot import OneHotEncoder

from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
pd.set_option('display.max_columns', None)

In [2]:
data = pd.read_csv("G:\My Drive\Espol\Cuarto semestre\Bootcamp\modulo05\churn.csv")

In [3]:
data.head()

,Customer ID,Gender,Age,Under 30,Senior Citizen,Married,Dependents,Number of Dependents,Country,State,City,Zip Code,Latitude,Longitude,Population,Quarter,Referred a Friend,Number of Referrals,Tenure in Months,Offer,Phone Service,Avg Monthly Long Distance Charges,Multiple Lines,Internet Service,Internet Type,Avg Monthly GB Download,Online Security,Online Backup,Device Protection Plan,Premium Tech Support,Streaming TV,Streaming Movies,Streaming Music,Unlimited Data,Contract,Paperless Billing,Payment Method,Monthly Charge,Total Charges,Total Refunds,Total Extra Data Charges,Total Long Distance Charges,Total Revenue,Satisfaction Score,Customer Status,Churn Label,Churn Score,CLTV,Churn Category,Churn Reason
0,8779-QRDMV,Male,78,No,Yes,No,No,0,United States,California,Los Angeles,90022,34.023810,-118.156582,68701,Q3,No,0,1,NaN,No,0.00,No,Yes,DSL,8,No,No,Yes,No,No,Yes,No,No,Month-to-Month,Yes,Bank Withdrawal,39.65,39.65,0.00,20,0.00,59.65,3,Churned,Yes,91,5433,Competitor,Competitor offered more data
1,7495-OOKFY,Female,74,No,Yes,Yes,Yes,1,United States,California,Los Angeles,90063,34.044271,-118.185237,55668,Q3,Yes,1,8,Offer E,Yes,48.85,Yes,Yes,Fiber Optic,17,No,Yes,No,No,No,No,No,Yes,Month-to-Month,Yes,Credit Card,80.65,633.30,0.00,0,390.80,1024.10,3,Churned,Yes,69,5302,Competitor,Competitor made better offer
2,1658-BYGOY,Male,71,No,Yes,No,Yes,3,United States,California,Los Angeles,90065,34.108833,-118.229715,47534,Q3,No,0,18,Offer D,Yes,11.33,Yes,Yes,Fiber Optic,52,No,No,No,No,Yes,Yes,Yes,Yes,Month-to-Month,Yes,Bank Withdrawal,95.45,1752.55,45.61,0,203.94,1910.88,2,Churned,Yes,81,3179,Competitor,Competitor made better offer
3,4598-XLKNJ,Female,78,No,Yes,Yes,Yes,1,United States,California,Inglewood,90303,33.936291,-118.332639,27778,Q3,Yes,1,25,Offer C,Yes,19.76,No,Yes,Fiber Optic,12,No,Yes,Yes,No,Yes,Yes,No,Yes,Month-to-Month,Yes,Bank Withdrawal,98.50,2514.50,13.43,0,494.00,2995.07,2,Churned,Yes,88,5337,Dissatisfaction,Limited range of services
4,4846-WHAFZ,Female,80,No,Yes,Yes,Yes,1,United States,California,Whittier,90602,33.972119,-118.020188,26265,Q3,Yes,1,37,Offer C,Yes,6.33,Yes,Yes,Fiber Optic,14,No,No,No,No,No,No,No,Yes,Month-to-Month,Yes,Bank Withdrawal,76.50,2868.15,0.00,0,234.21,3102.36,2,Churned,Yes,67,2793,Price,Extra data charges


Realizar análisis exploratorio del dataset. 
Esto con el objetivo de entender los datos y tomar decisiones en cuanto a ellos que puedan ser beneficiosas para la solución del problema.
Hacer Data Splitting. Si usan GridSearchCV o RandomizedSearchCV, no es necesario partir el data set en training/validation/testing,
sino solamente en training/testing, ya que estas herramientas realizan cross-validation, lo cual (entre otras cosas) 
implica que toman parte del training set para validación.

In [4]:
GridSearchCV o RandomizedSearchCV

SyntaxError: invalid syntax (226409224.py, line 1)

In [6]:
data.info()
# Your code here
ColumnsDrop=["Customer ID","Under 30","Senior Citizen","Country","State","City","Latitude","Longitude","Quarter"]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 50 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Gender                             7043 non-null   object 
 2   Age                                7043 non-null   int64  
 3   Under 30                           7043 non-null   object 
 4   Senior Citizen                     7043 non-null   object 
 5   Married                            7043 non-null   object 
 6   Dependents                         7043 non-null   object 
 7   Number of Dependents               7043 non-null   int64  
 8   Country                            7043 non-null   object 
 9   State                              7043 non-null   object 
 10  City                               7043 non-null   object 
 11  Zip Code                           7043 non-null   int64

In [16]:
data["Streaming TV"].value_counts()

Streaming TV
No     4336
Yes    2707
Name: count, dtype: int64

In [17]:
data["Offer"]

0           NaN
1       Offer E
2       Offer D
3       Offer C
4       Offer C
         ...   
7038        NaN
7039    Offer C
7040        NaN
7041        NaN
7042        NaN
Name: Offer, Length: 7043, dtype: object

In [12]:
data["Tenure in Months"].value_counts()

Tenure in Months
1     613
72    362
2     238
3     200
4     176
     ... 
38     59
28     57
39     56
44     51
36     50
Name: count, Length: 72, dtype: int64

In [8]:
data["Number of Referrals"].value_counts()

Number of Referrals
0     3821
1     1086
5      264
3      255
7      248
9      238
2      236
4      236
10     223
6      221
8      213
11       2
Name: count, dtype: int64

In [10]:
correlation = data['Referred a Friend'].corr(data["Number of Referrals"])
print("Correlación entre 'Total Charges' y 'Monthly Charge':", correlation)

ValueError: could not convert string to float: 'No'

In [ ]:
### ver relacion las variables con el targer (?)
data["Referred a Friend"].value_counts()

In [ ]:
data["Churn Label"].value_counts(normalize="True")

35  Paperless Billing                  7043 non-null   object 
 36  Payment Method                     7043 non-null   object 
 37  Monthly Charge                     7043 non-null   float64
 38  Total Charges                      7043 non-null   float64
 39  Total Refunds                      7043 non-null   float64
 40  Total Extra Data Charges           7043 non-null   int64  
 41  Total Long Distance Charges        7043 non-null   float64
 42  Total Revenue                      7043 non-null   float64
 43  Satisfaction Score                 7043 non-null   int64  
 44  Customer Status                    7043 non-null   object 
 45  Churn Label                        7043 non-null   object 

In [ ]:
data["Paperless Billing"].value_counts()

In [ ]:
data["Payment Method"].value_counts()

In [ ]:
data.head()

In [ ]:
data["Total Refunds"].value_counts()

In [ ]:
value_counts = data["Total Refunds"].value_counts()

# Crea un nuevo DataFrame agrupando los valores 0 y los diferentes de 0
grouped_data = pd.DataFrame({
    'Category': ['0', 'Different from 0'],
    'Count': [value_counts.loc[0], value_counts[value_counts.index != 0].sum()]
})

# Muestra el resultado
print(grouped_data)

In [ ]:
data[(data["Total Refunds"]!=0) & (data["Churn Label"]=="Yes")]

In [ ]:
data["Total Extra Data Charges"].value_counts()

In [ ]:
value_counts = data["Total Extra Data Charges"].value_counts()

# Convierte el resultado de value_counts a un DataFrame para facilitar la visualización
df = value_counts.reset_index()
df.columns = ['Total Extra Data Charges', 'Count']

In [ ]:
df

In [ ]:
plt.figure(figsize=(15, 6))  # Ajusta el valor 15 para hacerla aún más ancha si lo necesitas
plt.bar(df['Total Extra Data Charges'], df['Count'], color='skyblue')
plt.xlabel('Total Extra Data Charges')
plt.ylabel('Count')
plt.title('Distribution of Total Extra Data Charges')
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
data["Total Long Distance Charges"].value_counts() ##Relevante


In [ ]:
data["Total Revenue"].describe()

In [ ]:
data["Satisfaction Score"].value_counts()

In [ ]:
data[(data["Satisfaction Score"]==5) & (data["Churn Label"]=="Yes")]